In [1]:
import torch
import torch.nn as nn
import torch.utils.data as data
import torchvision
import torchvision.transforms as transforms
from torch.nn import functional as F

import numpy as np
import os
import csv
import random

In [2]:
BATCH_SIZE = 16
NUM_EPOCHS = 2
NUM_MODELS = 5

path = 'sjtu-m3dv-medical-3d-voxel-classification'
train_path = path+'/train_val'
test_path = path+'/test'

In [3]:
# 图像采样压缩
def compress(org):
    res = np.zeros((32,32,32))
    for i in range(32):
        for j in range(32):
            for k in range(32):
                res[i,j,k] = (org[3*i:3*i+2, 3*j:3*j+2, 3*k:3*k+2].sum()) / 27
    return res
# 生成随机数
def random_list(l,h,n):
    res = []
    for i in range(n):
        res.append(random.randint(l,h))
    return res

In [ ]:
# 打开训练集结果文件
train_res = []
with open(path+'/train_val.csv') as f:
    next(f)
    f_csv = csv.reader(f)
    for row in f_csv:
        train_res.append(float(row[1]))
f.close()
# 打开训练集数据文件
train_npz = os.listdir(train_path)
train_num = len(train_npz)

def open_train_seg():
    train_voxel = []
    train_seg = []
    for i in range(train_num):
        file = np.load(train_path+'/'+train_npz[i])
        seg = file['seg'].astype(np.float)*270
        t = random_list(0,4,3)+random_list(0,3,3)
        seg = compress(seg[t[0]:t[0]+96,t[1]:t[1]+96,t[2]:t[2]+96])
        seg = np.rot90(seg,t[3],(0,1))
        seg = np.rot90(seg,t[4],(0,2))
        seg = np.rot90(seg,t[5],(1,2))
        train_seg.append((seg,train_res[i]))
    # 加载训练集 
    train_seg_loader = data.DataLoader(train_seg,
                                       batch_size=BATCH_SIZE,
                                       shuffle=True,
                                       drop_last=True) 
    return train_seg_loader

In [5]:
# 打开测试集文件
test_npz = os.listdir(test_path)
test_num = len(test_npz)
def open_test_seg():
    test_seg = []
    for i in range(test_num):
        file = np.load(test_path+'/'+test_npz[i])
        seg = file['seg'].astype(np.float)*270
        t = random_list(0,4,3)+random_list(0,3,3)
        seg = compress(seg[t[0]:t[0]+96,t[1]:t[1]+96,t[2]:t[2]+96])
        seg = np.rot90(seg,t[3],(0,1))
        seg = np.rot90(seg,t[4],(0,2))
        seg = np.rot90(seg,t[5],(1,2))
        test_seg.append(seg)
    # 加载测试集
    test_seg_loader = data.DataLoader(test_seg, 
                                      batch_size=BATCH_SIZE,
                                      shuffle=True,
                                      drop_last=True) 
    return(test_seg_loader)

In [6]:
class DenseLayer(nn.Sequential):
    def __init__(self,in_channel,growth_rate,bn_size,drop_rate): 
        super(DenseLayer,self).__init__()
        self.add_module('norm1',nn.BatchNorm3d(in_channel)),
        self.add_module('relu1',nn.ReLU(inplace=True)),
        self.add_module('conv1',nn.Conv3d(in_channel,bn_size*growth_rate,
                                          kernel_size=1,stride=1,bias=False)),
        self.add_module('norm2',nn.BatchNorm3d(bn_size*growth_rate)),
        self.add_module('relu2',nn.ReLU(inplace=True))
        self.add_module('conv2',nn.Conv3d(bn_size*growth_rate,growth_rate,
                                          kernel_size=3,stride=1,padding=1,bias=False))
        self.drop_rate = drop_rate
    def forward(self,x):
        new_feature = super(DenseLayer,self).forward(x)
        if(self.drop_rate>0):
            new_feature = F.dropout3d(new_feature,p=self.drop_rate,training=self.training)
        return torch.cat([x,new_feature],1) 

class DenseBlock(nn.Sequential):
    def __init__(self,layer_num,in_channel,bn_size,growth_rate,drop_rate):
        super(DenseBlock,self).__init__()
        for i in range(layer_num):
            layer = DenseLayer(in_channel+i*growth_rate,growth_rate,bn_size,drop_rate)
            self.add_module('denselayer%d'%(i+1),layer)

class Transition(nn.Sequential):
    def __init__(self,in_channel,zip_ratio=0.5):
        super(Transition,self).__init__()
        self.add_module('norm',nn.BatchNorm3d(in_channel))
        self.add_module('relu',nn.ReLU(inplace=True))
        self.add_module('conv',nn.Conv3d(in_channel,int(in_channel*zip_ratio),
                                         kernel_size=1,stride=1,bias=False))
        self.add_module('pool',nn.AvgPool3d(kernel_size=2,stride=2))

#卷积池化层
def FeatureLayer(in_channel,out_channel):
    layer = nn.Sequential(
            nn.Conv3d(in_channel,out_channel,
                      kernel_size=3,stride=1,padding=3,bias=False),
            nn.BatchNorm3d(num_features=out_channel), 
            nn.ReLU(inplace=True),
            nn.AvgPool3d(kernel_size=3,stride=2,padding=1))
    return layer

class DenseNet(nn.Module):
    
    def __init__(self,growth_rate=32,block_config=(4,4,4),init_channel_num=64,
                 bn_size=4,drop_rate=0):
        super(DenseNet,self).__init__()
        self.feature_layer = FeatureLayer(1,init_channel_num)
        channel_num = init_channel_num
        for i,layer_num in enumerate(block_config):
            block = DenseBlock(layer_num=layer_num,in_channel=channel_num,
                               bn_size=bn_size,growth_rate=growth_rate,
                               drop_rate=drop_rate)
            self.feature_layer.add_module('denseblock%d'%(i+1),block)
            channel_num = channel_num + layer_num*growth_rate
            if(i!=len(block_config)-1):
                trans = Transition(channel_num,0.5)
                self.feature_layer.add_module('transition%d'%(i+1),trans)
                channel_num = int(0.5*channel_num)
        self.feature_layer.add_module('norm5',nn.BatchNorm3d(channel_num))
        self.feature_layer.add_module('relu5',nn.ReLU(inplace=True))
        self.feature_layer.add_module('avgpool5',
                                      nn.AvgPool3d(kernel_size=3,stride=2))
        self.classifier = nn.Linear(channel_num,1) 
        self.sigmoid = torch.sigmoid
        
    def forward(self,x):
        features = self.feature_layer(x)
        out = features.view(len(x),-1)
        out = self.classifier(out)
        out = self.sigmoid(out)
        out = out.view(len(out))
        return out

In [7]:
model_seg = DenseNet(drop_rate=0.3)

optimizer_seg = torch.optim.SGD(model_voxel.parameters(),lr=0.01)
criterion_seg = nn.MultiLabelSoftMarginLoss()

In [8]:
# 训练 
for i in range(NUM_MODELS):
    model_seg.train()
    for epoch in range(NUM_EPOCHS):
        train_seg_loader = open_train_seg()
        optimizer_seg.zero_grad()
        for images,labels in train_seg_loader:
            images = images.view(BATCH_SIZE,1,32,32,32)
            outputs = model_seg(images.float())
            labels = labels.view(BATCH_SIZE,1)
            outputs = outputs.view(BATCH_SIZE,1)
            loss = criterion_seg(outputs,labels)
            loss.backward()
            optimizer_seg.step()
    name = 'try_3_seg_model_' + str(i+1) + '.pkl'
    torch.save(model_seg,name)

In [10]:
# 测试
model = torch.load('try_0_voxel_model.pkl')
pred = []
for i in range(NUM_MODELS):
    name = 'try_3_seg_model_' + str(i+1) + '.pkl'
    model = torch.load('name')
    for j in range(NUM_MODELS):
        test_seg_loader = open_test_seg()
        model_seg.eval()
        pred_seg = torch.ones(0)
        for images in test_seg_loader:
            images = images.view(BATCH_SIZE,1,32,32,32)
            with torch.no_grad():
                outputs = model_seg(images.float())
            pred_seg = torch.cat([pred_seg,outputs]) 
            
        pred.append(pred_seg)

out = pred[0]
for i in range(1,len(pred)):
    out = out + pred[i]
    
out = out/len(pred)

In [ ]:
np.savetxt('try_3.csv', out, delimiter = ',')